In [ ]:
!pip install gradio -q -U

In [ ]:
!pip install pydub --quiet

In [ ]:
from pydub import AudioSegment
import os
import math
from transformers import pipeline
import gradio as gr


In [ ]:
def split_audio(input_file, output_folder, duration):
    audio = AudioSegment.from_mp3(input_file)
    total_length = len(audio)
    num_parts = math.ceil(total_length / (duration * 1000))

    for i in range(num_parts):
        start = i * duration * 1000
        end = (i + 1) * duration * 1000
        split_audio = audio[start:end]
        output_path = os.path.join(output_folder, f"part_{i+1}.mp3")
        split_audio.export(output_path, format="mp3")
        print(f"Exported {output_path}")

In [ ]:
def transcribir(file):
  pipe = pipeline("automatic-speech-recognition", "openai/whisper-large-v3",return_timestamps=True,device=0)
  os.mkdir("Salida")
  split_audio(file, "Salida", 600)
  f = open("Guión.docx","a")
  for file in sorted(os.listdir("Salida")):
      file_path = os.path.join("Salida", file)
      print(file_path)
      f.write((pipe(file_path)["text"]))
  f.close()
  return f

In [ ]:
demo = gr.Interface(
    fn=transcribir,
    inputs=gr.File(),
    outputs=gr.File(),
    allow_flagging='never'
)
demo.launch(share=True,debug=True)